In [1]:
import os
import sys

#sys.path.insert(0, os.path.abspath('../genophenocorr'))
from genophenocorr import *

In [2]:
allPatients = Cohort('../phenopackets/phenopacketGRIN2B/*.json', ref = 'hg38')

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle


WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described 

In [3]:
SplitProts = allPatients.split_by_protein()
GRIN1 = SplitProts.get('ENSP00000360601')
GRIN2B = SplitProts.get('ENSP00000477455')
GRIN2A = SplitProts.get('ENSP00000332549')

In [4]:
import hpotk
from hpotk.ontology import Ontology
from hpotk.ontology.load.obographs import load_ontology
from hpotk.validate import ObsoleteTermIdsValidator

# to peek under the hood
import logging
hpotk.util.setup_logging(logging.DEBUG)

o = load_ontology('https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.json')

DEBUG:hpotk.util:Opening https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.json
2023-03-03 17:09:53,686 hpotk.util           DEBUG : Opening https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.json
DEBUG:hpotk.util:Using default encoding 'utf-8'
2023-03-03 17:09:53,687 hpotk.util           DEBUG : Using default encoding 'utf-8'
DEBUG:hpotk.util:Looks like a URL: https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.json
2023-03-03 17:09:53,688 hpotk.util           DEBUG : Looks like a URL: https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.json
DEBUG:hpotk.util:Downloading with timeout=30s
2023-03-03 17:09:53,688 hpotk.util           DEBUG : Downloading with timeout=30s
DEBUG:hpotk.util:Looks like un-compressed data
2023-03-03 17:09:53,810 hpotk.util           DEBUG : Looks like un-compressed data
DEBUG:hpotk.ontology.load.obographs._load:Extracting ontology terms
20

In [5]:
GRIN2B.list_all_phenotypes()

[['HP:0001249', 'Intellectual disability'],
 ['HP:0012758', 'Neurodevelopmental delay'],
 ['HP:0001263', 'Global developmental delay'],
 ['HP:0002342', 'Intellectual disability, moderate'],
 ['HP:0000729', 'Autistic behavior'],
 ['HP:0007359', 'Focal-onset seizure'],
 ['HP:0002197', 'Generalized-onset seizure'],
 ['HP:0000717', 'Autism'],
 ['HP:0001250', 'Seizure'],
 ['HP:0010864', 'Intellectual disability, severe'],
 ['HP:0100022', 'Abnormality of movement'],
 ['HP:0000023', 'Inguinal hernia'],
 ['HP:0001252', 'Hypotonia'],
 ['HP:0001999', 'Abnormal facial shape'],
 ['HP:0002827', 'Hip dislocation'],
 ['HP:0011097', 'Epileptic spasm'],
 ['HP:0001332', 'Dystonia'],
 ['HP:0011182', 'Interictal epileptiform activity'],
 ['HP:0011179', 'Beta-EEG'],
 ['HP:0011293', 'EEG with central sharp waves'],
 ['HP:0011193', 'EEG with focal spikes'],
 ['HP:0011296', 'EEG with temporal sharp waves'],
 ['HP:0002123', 'Generalized myoclonic seizure'],
 ['HP:0001344', 'Absent speech'],
 ['HP:0100660', 'Dy

In [6]:
Term = o.get_term('HP:0001290')
Term

Term(identifier=TermId(prefix="HP", id="0001290"), name="Generalized hypotonia", definition=Generalized muscular hypotonia (abnormally low muscle tone)., comment=None, is_obsolete=False, alt_term_ids="[]")

In [7]:
print('#' * 20 + ' Parents ' + '#' * 20)
for parent in hpotk.algorithm.get_ancestors(o, Term.identifier):
    term = o.get_term(parent)
    print(f"{term.identifier.value} - {term.name}")

#################### Parents ####################
HP:0001252 - Hypotonia
HP:0033127 - Abnormality of the musculoskeletal system
HP:0003808 - Abnormal muscle tone
HP:0000118 - Phenotypic abnormality
HP:0003011 - Abnormality of the musculature
HP:0011804 - Abnormal muscle physiology
HP:0000001 - All


In [10]:
for d in GRIN2B.all_phenotypes_d:
    print (len(GRIN2B.all_phenotypes_d[d].list_ancestors()))
    


6
5
6
7
5
5
5
6
4
7
4
9
6
5
16
6
5
8
9
12
10
12
9
10
5
6
5
8
11
8
7
19
7
5
6
4
9
6
5
5
4
6
7
6
5
7
7


In [11]:
from hpotk.validate import AnnotationPropagationValidator
import re

HPOs = []
for hpoid, hpoval in GRIN2B.list_all_phenotypes():
    HPOs.append(o.get_term(hpoid))

descend = []
descendents = hpotk.algorithm.get_descendants(o, 'HP:0001250')
print(descendents)
for d in descendents:
    descend.append(d.value)
    
ap = AnnotationPropagationValidator(o)
results = ap.validate(HPOs)
print(results)
for r in results.results:
    child = re.findall(r"HP:\d{7}", r.message)[0]
    if child in descend:
        print(f"{child} is a descendent of HP:0001250")

print(type(results))

frozenset({TermId(prefix="HP", id="0031165"), TermId(prefix="HP", id="0032878"), TermId(prefix="HP", id="0032744"), TermId(prefix="HP", id="0032829"), TermId(prefix="HP", id="0032821"), TermId(prefix="HP", id="0007334"), TermId(prefix="HP", id="0032715"), TermId(prefix="HP", id="0032761"), TermId(prefix="HP", id="0032690"), TermId(prefix="HP", id="0032844"), TermId(prefix="HP", id="0033346"), TermId(prefix="HP", id="0032678"), TermId(prefix="HP", id="0010820"), TermId(prefix="HP", id="0032688"), TermId(prefix="HP", id="0032837"), TermId(prefix="HP", id="0032852"), TermId(prefix="HP", id="0032807"), TermId(prefix="HP", id="0032673"), TermId(prefix="HP", id="0032730"), TermId(prefix="HP", id="0032810"), TermId(prefix="HP", id="0032722"), TermId(prefix="HP", id="0032919"), TermId(prefix="HP", id="0032734"), TermId(prefix="HP", id="0032704"), TermId(prefix="HP", id="0032711"), TermId(prefix="HP", id="0032765"), TermId(prefix="HP", id="0011145"), TermId(prefix="HP", id="0020210"), TermId(pr

In [12]:
TempTerm = o.get_term('HP:0006010')

obso = ObsoleteTermIdsValidator(o)
inputs = [
    TempTerm
]

results = obso.validate(inputs)
print(results)

ValidationResults(is_ok=True, n_results=0)


In [17]:
for var in GRIN2B.all_variants_l:
    print(var.variant_string)
    result_chi2 = GRIN2B.run_stats(is_var_match, is_not_var_match, var.variant, var.variant, percent_patients=20, combine_like_hpos = True)
    print(result_chi2.head(5))

chr12 g.13569988G>A
                                       1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0012758 - Neurodevelopmental delay       1.0        0.0       9.0   
HP:0001249 - Intellectual disability        1.0        0.0      31.0   
HP:0000729 - Autistic behavior              0.0        1.0      11.0   
HP:0001250 - Seizure                        0.0        1.0      13.0   

                                       2 w/o hpo      pval  adjusted pval  
HP:0012758 - Neurodevelopmental delay       24.0  0.294118            1.0  
HP:0001249 - Intellectual disability         2.0  1.000000            1.0  
HP:0000729 - Autistic behavior              22.0  1.000000            1.0  
HP:0001250 - Seizure                        20.0  1.000000            1.0  
chr12 g.13616477A>G
                                       1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0000729 - Autistic behavior              1.0        0.0      10.0   
HP:0001250 - Seizure                        1.0        0.0      12.0   
HP:

                                       1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0001250 - Seizure                        0.0        5.0      13.0   
HP:0001249 - Intellectual disability        4.0        1.0      28.0   
HP:0012758 - Neurodevelopmental delay       2.0        3.0       8.0   
HP:0000729 - Autistic behavior              2.0        3.0       9.0   

                                       2 w/o hpo      pval  adjusted pval  
HP:0001250 - Seizure                        16.0  0.131717       0.526867  
HP:0001249 - Intellectual disability         1.0  0.276292       0.552585  
HP:0012758 - Neurodevelopmental delay       21.0  0.618121       0.824162  
HP:0000729 - Autistic behavior              20.0  1.000000       1.000000  
chr12 g.13615221T>C
                                       1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0001249 - Intellectual disability        1.0        0.0      31.0   
HP:0012758 - Neurodevelopmental delay       0.0        1.0      10.0   
HP:0000729 - Autistic b

                                       1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0012758 - Neurodevelopmental delay       1.0        0.0       9.0   
HP:0001249 - Intellectual disability        1.0        0.0      31.0   
HP:0000729 - Autistic behavior              0.0        1.0      11.0   
HP:0001250 - Seizure                        0.0        1.0      13.0   

                                       2 w/o hpo      pval  adjusted pval  
HP:0012758 - Neurodevelopmental delay       24.0  0.294118            1.0  
HP:0001249 - Intellectual disability         2.0  1.000000            1.0  
HP:0000729 - Autistic behavior              22.0  1.000000            1.0  
HP:0001250 - Seizure                        20.0  1.000000            1.0  
